In [1]:
import numpy as np
import pandas as pd
import math

In [22]:
def dist(x, y, p):
        distance = 0
        for i in range(x.shape[0]):
            distance+=np.abs(x[i]-y[i])**p
        return distance**(1/p)
class kNN:
    def __init__(self, k, p, classes):
        self.k = k
        self.p = p
        self.classes = classes
    def fit(self,X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
       
    def predict(self,X_test):
        labels = []
        for vector in X_test:
            distance = [[self.y_train[i],dist(vector, self.X_train[i], self.p)]for i in range(self.X_train.shape[0])]
            distance = sorted(distance,key = lambda x: x[1])
            classes_around = np.zeros(self.classes)
            for i in range(self.k):
                classes_around[int(distance[i][0])]+=1
            labels.append(sorted(zip(classes_around, range(self.classes)), key=lambda x:x[0], reverse=True)[0][1])
        return labels

In [41]:
df = pd.read_csv('../datasets/csgo_task_pred.csv')
print(df["bomb_planted"].value_counts())
bomb_planted = df[df["bomb_planted"] == 1]
bomb_not_planted  = df[df["bomb_planted"] == 0]
from sklearn.utils import resample
bomb_not_planted = resample(bomb_not_planted,
             replace=True,
             n_samples=len(bomb_planted),
             random_state=42)
bomb_not_planted.shape
df1 = pd.concat([bomb_not_planted, bomb_planted])
df1 = df1.sample(n=1000)
y = df1["bomb_planted"]
X = df1.drop(["bomb_planted"], axis=1)

0    108537
1     13662
Name: bomb_planted, dtype: int64


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train_minmax = pd.DataFrame(min_max_scaler.fit_transform(X_train))
X_test_minmax = pd.DataFrame(min_max_scaler.fit_transform(X_test))

In [44]:
model = kNN(5,1,2)
X_train_minmax = np.array(X_train_minmax, dtype=np.float64)
X_test_minmax = np.array(X_test_minmax, dtype=np.float64)
y_train= np.array(y_train, dtype=np.float64)
len(X_train_minmax)
model.fit(X_train_minmax, y_train)
y_pred3 = model.predict(X_test_minmax)
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.98      0.80      0.88       113
           1       0.79      0.98      0.87        87

    accuracy                           0.88       200
   macro avg       0.88      0.89      0.87       200
weighted avg       0.90      0.88      0.88       200



In [11]:
print(y_test)

[0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 0 0 1 1]


In [66]:
def confusion_matrix(y, y_pred):
  clas = len(np.unique(y))
  conf_matrix = np.zeros((clas, clas))
  for i in range(len(y)):
    conf_matrix[y.iloc[i]][y_pred[i]] += 1
  return conf_matrix

In [54]:
def acurracy(y, y_pred):
    m = confusion_matrix(y, y_pred)
    return (m[0][0]+m[1][1])/len(y)

In [55]:
def precision(y, y_pred):
    m = confusion_matrix(y, y_pred)
    return m[1][1]/(m[1][1]+m[0][1])

In [56]:
def recall(y, y_pred):
    m = confusion_matrix(y, y_pred)
    return m[1][1]/(m[1][1]+m[1][0])

In [57]:
def f1(y, y_pred):
    return 2/((1/precision(y, y_pred))+(1/recall(y, y_pred)))

In [68]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.98      0.80      0.88       113
           1       0.79      0.98      0.87        87

    accuracy                           0.88       200
   macro avg       0.88      0.89      0.87       200
weighted avg       0.90      0.88      0.88       200



In [69]:
y_test.iloc[0]

0

In [70]:
print(acurracy(y_test, y_pred3))
print(precision(y_test, y_pred3))
print(recall(y_test, y_pred3))
print(f1(y_test, y_pred3))

0.875
0.7870370370370371
0.9770114942528736
0.8717948717948718
